In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import  create_retrieval_chain
from langchain.prompts.chat import ChatPromptTemplate
from langchain_ollama import ChatOllama

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
web_paths = [
    'https://python.langchain.com/docs/introduction/',
    'https://docs.smith.langchain.com/',
    'https://www.ibm.com/think/topics/ai-agents'
]

loader = WebBaseLoader(web_paths=web_paths)
documents = loader.load()

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)

In [15]:
documents

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1\uf8ffüí¨SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with cha

In [16]:
embedding = OllamaEmbeddings(model='mxbai-embed-large:335m')

In [17]:
vectordb = Chroma.from_documents(documents=documents, embedding=embedding)

In [18]:
vectordb.similarity_search('what is langsmith')

[Document(metadata={'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en', 'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}, page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'),
 Document(metadata={'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en', 'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}, page_content="LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.\nObservability‚Äã\nObservability is important for any software application, but especially so for LL

In [24]:
template = """Answer the question based only on the following context:

{context}

Question: {input}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are an helpful assistant'),
        ('user', template)
    ]
)

In [29]:
relevant_docs = retriever.invoke('what is agent')

In [30]:
prompt.invoke({'input' : 'what is ai', 'context' : relevant_docs})

ChatPromptValue(messages=[SystemMessage(content='You are an helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content="Answer the question based only on the following context:\n\n[Document(metadata={'description': 'An artificial intelligence (AI) agent refers to a system or program that is capable of autonomously performing tasks on behalf of a user or another system.', 'language': 'en', 'source': 'https://www.ibm.com/think/topics/ai-agents', 'title': 'What Are AI Agents? | IBM'}, page_content='What Are AI Agents? | IBM\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n                        \\n\\n\\n\\n  \\n    What are AI agents?\\n\\n\\n\\n\\n\\n\\n    \\n\\n\\n                    \\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n              

In [19]:
retriever = vectordb.as_retriever()

In [20]:
llm = ChatOllama(model='llama3.2:3b')

In [20]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [21]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [23]:
retrieval_chain.invoke({'input' : 'what is ai agents'})

{'input': 'what is ai agents',
 'context': [Document(metadata={'description': 'An artificial intelligence (AI) agent refers to a system or program that is capable of autonomously performing tasks on behalf of a user or another system.', 'language': 'en', 'source': 'https://www.ibm.com/think/topics/ai-agents', 'title': 'What Are AI Agents? | IBM'}, page_content='What Are AI Agents? | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                        \n\n\n\n  \n    What are AI agents?\n\n\n\n\n\n\n    \n\n\n                    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                    Artificial Intelligence\n                                \n\n\n\n\n\n\n                    \n\n\n\n  \n    3 July 2024\n\n\n\n\n\n\n    \n\n\n                \n\n\n\n\n\n\n\n\n\n\n\n\n                Link copied\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

### Adding Chat History

In [60]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain, RetrievalQA
from langchain.embeddings import OpenAIEmbeddings

In [61]:
memory = ConversationBufferMemory()

In [62]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),  # 👈 Keeps past conversations
    ("human", "{question}"),  # 👈 Takes new user input
])

In [63]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Uses simple document stuffing method
    retriever=retriever,
    memory=memory
)

In [64]:
# First question
user_input = "Tell me about LangChain."
response = rag_chain.run(user_input)
print(response)  # ✅ AI fetches info & answers

C:\Users\TALHA PC\AppData\Local\Temp\ipykernel_11780\3011989571.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain.run(user_input)


LangChain is a framework for developing applications powered by large language models (LLMs). It simplifies every stage of the LLM application lifecycle, making it easier to build and deploy complex applications using these powerful AI tools.

LangChain aims to provide a comprehensive platform for developers to create, train, and run large language model-based applications. By providing a range of resources, guides, and pre-built components, LangChain makes it easier for users to get started with building their own LLM-powered applications.

Some key aspects of LangChain include:

*   A set of pre-built components that can be easily integrated into applications
*   A framework for building cognitive architectures using chains, agents, and retrieval strategies
*   Integration packages for popular LLMs like OpenAI and Anthropic
*   Documentation and tutorials to help users get started

Overall, LangChain is designed to make it easier for developers to build and deploy complex language mo

In [65]:
# Follow-up question (AI should remember context)
user_input2 = "Can you explain its use cases?"
response2 = rag_chain.run(user_input2)
print(response2)

AI agents have a wide range of applications across various industries. Here are some common use cases:

1. **Virtual Assistants**: AI agents can be integrated into websites and apps to serve as virtual assistants, providing customer support, answering frequently asked questions, and helping users with tasks.
2. **Conversational Interfaces**: AI agents can power conversational interfaces for chatbots, voice assistants, and other human-computer interactions, enabling businesses to provide 24/7 support and engage with customers more effectively.
3. **Customer Experience**: AI agents can be used to personalize customer experiences, offer product recommendations, and provide personalized content based on user behavior and preferences.
4. **Healthcare**: AI agents can assist in medical diagnosis, treatment planning, and patient care, freeing up healthcare professionals to focus on high-value tasks.
5. **Automation and Process Optimization**: AI agents can automate repetitive tasks, optimize 

### Using another technique

In [45]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain.prompts.chat import ChatPromptTemplate

In [38]:
system_prompt = (
    "Give a chat history and the latest user question"
    "which might reference context in the chat history,"
    "formulate a stadalone question which can be understood"
    "without the chat history. Do Not answer the question"
    'just reformulate it if needed and otherwise return it as is.'
)

In [39]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human', '{input}')
    ]
)

In [40]:
memory = []

In [41]:
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

In [42]:
history_aware_retriever.invoke({'input' : 'what is ai agents', 'chat_history' : memory})

[Document(metadata={'description': 'An artificial intelligence (AI) agent refers to a system or program that is capable of autonomously performing tasks on behalf of a user or another system.', 'language': 'en', 'source': 'https://www.ibm.com/think/topics/ai-agents', 'title': 'What Are AI Agents? | IBM'}, page_content='What Are AI Agents? | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                        \n\n\n\n  \n    What are AI agents?\n\n\n\n\n\n\n    \n\n\n                    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                    Artificial Intelligence\n                                \n\n\n\n\n\n\n                    \n\n\n\n  \n    3 July 2024\n\n\n\n\n\n\n    \n\n\n                \n\n\n\n\n\n\n\n\n\n\n\n\n                Link copied\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n

In [43]:
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000021D4095EF30>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChun

In [54]:
template = """Answer the question based only on the following context:

{context}

Question: {input}
"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human', template)
    ]
)

In [55]:
document_chain = create_stuff_documents_chain(llm, qa_prompt)

In [56]:
rag_chain = create_retrieval_chain(history_aware_retriever, document_chain)

In [62]:
from langchain_core.messages import HumanMessage, AIMessage

In [63]:
chat_history = []

question = 'what is ai agents'

response = rag_chain.invoke({'input' : question, 'chat_history' : chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response['answer'])
    ]
)

In [65]:
question = 'what is ai'

response = rag_chain.invoke({'input' : question, 'chat_history' : chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response['answer'])
    ]
)

In [66]:
chat_history

[HumanMessage(content='what is ai agents', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Here\'s a reformulated version of the question that can be understood without referencing the chat history:\n\n"What are AI agents?"', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is ai', additional_kwargs={}, response_metadata={}),
 AIMessage(content='According to the context provided by the industry newsletter, an AI agent refers to a system or program that autonomously performs tasks on behalf of a user or another system, designed to utilize available tools and workflow to complete complex tasks.', additional_kwargs={}, response_metadata={})]

In [68]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [69]:
store = {}

def get_session_memory(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    else:
        return store[session_id]

In [70]:
runnable = RunnableWithMessageHistory(
    rag_chain,
    get_session_memory,
    input_messages_key='input',
    history_messages_key='chat_history',
    output_messages_key='answer'
)

In [73]:
runnable.invoke(
    {"input" : "what is ai agents"},
    config = {'configurable' : {'session_id' : 'chat1'}}
)

{'input': 'what is ai agents',
 'chat_history': [],
 'context': [Document(metadata={'description': 'An artificial intelligence (AI) agent refers to a system or program that is capable of autonomously performing tasks on behalf of a user or another system.', 'language': 'en', 'source': 'https://www.ibm.com/think/topics/ai-agents', 'title': 'What Are AI Agents? | IBM'}, page_content='What Are AI Agents? | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                        \n\n\n\n  \n    What are AI agents?\n\n\n\n\n\n\n    \n\n\n                    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                    Artificial Intelligence\n                                \n\n\n\n\n\n\n                    \n\n\n\n  \n    3 July 2024\n\n\n\n\n\n\n    \n\n\n                \n\n\n\n\n\n\n\n\n\n\n\n\n                Link copied\n            \n\n\n\n\n\n\n\n

In [75]:
runnable.invoke(
    {"input" : "and what are its usecases"},
    config = {'configurable' : {'session_id' : 'chat1'}}
)

{'input': 'and what are its usecases',
 'chat_history': [HumanMessage(content='what is ai agents', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Is an AI agent a system that can autonomously perform tasks on behalf of a user or another system by designing its workflow and utilizing available tools?', additional_kwargs={}, response_metadata={})],
 'context': [Document(metadata={'description': 'An artificial intelligence (AI) agent refers to a system or program that is capable of autonomously performing tasks on behalf of a user or another system.', 'language': 'en', 'source': 'https://www.ibm.com/think/topics/ai-agents', 'title': 'What Are AI Agents? | IBM'}, page_content='AI agents can encompass a wide range of functionalities beyond natural language processing including decision-making, problem-solving, interacting with external environments and executing actions.\nThese agents can be deployed in various applications to solve complex tasks in various enterprise con